In [19]:
import scrapy
from bs4 import BeautifulSoup
import os
import requests
import time
import re
from pathlib import Path
import pandas as pd

In [21]:
titlelist, companylist, locationlist, salarylist, summarylist, sublinklist, fullsummary = [],[],[],[],[],[],[]
columns = ['job_title', 'company_name', 'location', 'salary', 'summary', 'sublink', 'fullsummary']
output_df = pd.DataFrame(columns=columns)
start_time = time.time
max_results_per_city = 3000
from selenium import webdriver
for start in range(0, max_results_per_city, 10):
    page = requests.get(
        'https://www.indeed.com/jobs?q=data+analyst&start='+ str(start))
    soup = BeautifulSoup(page.text, 'html.parser')
    jobsearch = soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'})

    titlelist = []
    for i in jobsearch:
        titles = i.find_all('div',attrs={'class':'title'})
        for j in titles:
            temp1 = j.text
        titlelist.append(temp1.strip())

    companylist = []
    for i in jobsearch:
        companies = i.find_all('span',attrs={'class':'company'})
        for j in companies:
            temp2 = j.text
        companylist.append(temp2.strip())
    
    locationlist = []
    for i in jobsearch:
        locations = i.find_all('div',attrs={'class':'location'})
        for j in locations:
            temp3 = j.text
        try:
            locationlist.append(temp3.strip())
        except:
            locationlist.append('None')
        temp3 = 'None'
    
    jobsearch = soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'})
    salarylist = []
    for i in jobsearch:
        salaries = i.find_all('div',attrs={'class':'salarySnippet'})
        for j in salaries:
            temp4 = j.text
        salarylist.append(temp4.strip())
        temp4 = 'None'
    
    jobsearch = soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'})
    summarylist = []
    for i in jobsearch:
        summaries = i.find_all('div',attrs={'class':'summary'})
        for j in summaries:
            temp5 = j.text
        summarylist.append(temp5.strip())
    
    jobsearch = soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result'})
    sublinklist = []
    for i in jobsearch:
        sublinks = i.find_all('div',attrs={'class':'title'})
        for j in sublinks:
            for g in j.find_all('a'):
                temp6 = g.get('href')
                sublinklist.append('https://www.indeed.com'+str(temp6))

    fullsummary = []
    for sublink in sublinklist:
        page = requests.get(sublink)
        soup = BeautifulSoup(page.text, 'lxml')
        uls = soup.find_all('ul')
        temp7 = ''
        for ul in uls:
            lis = ul.find_all('li')
            for li in lis:
                temp7 = temp7 + li.text
        fullsummary.append([temp7])

    # Concatenate by columns
    title = pd.Series(list(titlelist))
    company_name = pd.Series(list(companylist))
    location = pd.Series(list(locationlist))
    salary = pd.Series(list(salarylist))
    summary = pd.Series(list(summarylist))
    sublink = pd.Series(list(sublinklist))
    fullsummary = pd.Series(fullsummary)
    sample_df = pd.concat([title, company_name, location, salary, summary, sublink, fullsummary], axis=1)
    sample_df.columns = columns
    output_df = pd.concat([output_df,sample_df], ignore_index=True)
    time.sleep(3)
print("--- %s seconds ---" % (time.time() - start_time))

In [22]:
output_df.shape

(2753, 7)

In [23]:
output_df.head()

,job_title,company_name,location,salary,summary,sublink,fullsummary
0,Visual Data Analyst,Sperry Rail Inc.,"Shelton, CT",$15 an hour,"Visual Data Analyst*. As a visual analyst, you...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,[Find JobsCompany ReviewsFind SalariesFind Res...
1,Data Analyst,AGI,"Virginia Beach, VA 23452 (North Central area)",None,Preferred understanding of Information Systems...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,[Find JobsCompany ReviewsFind SalariesFind Res...
2,Automotive Data Analyst,WHI Solutions Inc,"Rye Brook, NY 10573","$68,000 - $78,000 a year",Automotive Technical Support Analyst. Auto Car...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,[Find JobsCompany ReviewsFind SalariesFind Res...
3,Quality Data Analyst,AK Steel,"Butler, PA 16003",None,AK Steel is looking for an experienced Quality...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,[Find JobsCompany ReviewsFind SalariesFind Res...
4,Data Analyst,LeadingAge PA,"Mechanicsburg, PA",None,"Coordinating data collection, analysis and dis...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,[Find JobsCompany ReviewsFind SalariesFind Res...


In [30]:
output_df.to_csv('jobs.csv', encoding='utf-8')